In [83]:
# import necessary libraries
import os

from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

import pandas as pd

In [136]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///Hospital_Data.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# get all the classes
ed_triage = Base.classes.ED_Triage
ed_on_time = Base.classes.ED_on_time
ed_waiting = Base.classes.ED_waiting
ed_waiting_peer_group = Base.classes.ED_waiting_Peer_Group
hospitals = Base.classes.Hospitals
peer_group = Base.classes.Peer_Group

session = Session(engine)

# convert to dataframes
ed_triage_df = pd.read_sql(session.query(ed_triage).statement,session.bind)
ed_on_time_df = pd.read_sql(session.query(ed_on_time).statement,session.bind)
ed_waiting_df = pd.read_sql(session.query(ed_waiting).statement,session.bind)
ed_waiting_peer_group_df = pd.read_sql(session.query(ed_waiting_peer_group).statement,session.bind)
hospitals_df = pd.read_sql(session.query(hospitals).statement,session.bind)
peer_group_df = pd.read_sql(session.query(peer_group).statement,session.bind)
peer_group_df['Peer_Group_ID'] = peer_group_df['Peer_group_ID']

# merge dataframes/clean the data

merge = ed_on_time_df.merge(ed_triage_df[['Triage_ID','Triage_category','treatment_required_in']])

merge1 = hospitals_df.merge(peer_group_df[['Peer_Group_ID','Peer_group_name']])
merge2 = merge1.merge(ed_waiting_df[['Hospital_ID','Patient_cohort','No_of_presentations','Median_time_in_ED','Year']])
merge2['No_of_presentations_waiting'] = merge2['No_of_presentations']
merge2['Median_time_in_ED_waiting'] = merge2['Median_time_in_ED']

cleaned_df1 = merge[['Hospital_ID','Year','Triage_category','No_of_presentations','Percentage_of_patients_seen_on_time']]
cleaned_df1['No_of_presentations_ontime'] = cleaned_df1['No_of_presentations']

merge3 = peer_group_df.merge(ed_waiting_peer_group_df[['Peer_group_ID','Year','Patient_cohort','Avg_Wait_time']])
merge2 = merge2.merge(merge3[['Peer_group_ID','Patient_cohort','Avg_Wait_time']])

cleaned_df2 = merge2[['Hospital_ID','Hospital_Name','Latitude','Longitude','State', 'Sector', 'Year','Patient_cohort','Peer_group_name','No_of_presentations_waiting','Median_time_in_ED_waiting']]

joined_df = cleaned_df2.merge(cleaned_df1[['Hospital_ID','Year','Triage_category','No_of_presentations_ontime','Percentage_of_patients_seen_on_time']])


print(joined_df)

/Users/joshbrodrick/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


       Hospital_ID                        Hospital_Name   Latitude  \
0            H0014  The Children's Hospital at Westmead -33.801554   
1            H0014  The Children's Hospital at Westmead -33.801554   
2            H0014  The Children's Hospital at Westmead -33.801554   
3            H0014  The Children's Hospital at Westmead -33.801554   
4            H0014  The Children's Hospital at Westmead -33.801554   
...            ...                                  ...        ...   
144823       H0738               Alice Springs Hospital -23.705758   
144824       H0738               Alice Springs Hospital -23.705758   
144825       H0738               Alice Springs Hospital -23.705758   
144826       H0738               Alice Springs Hospital -23.705758   
144827       H0738               Alice Springs Hospital -23.705758   

         Longitude State  Sector  Year                  Patient_cohort  \
0       150.991759   NSW  Public  2020                    All patients   
1       150

In [133]:

print(merge3)



   Peer_group_ID                Peer_group_name Peer_Group_ID  Year  \
0              1                       National             1  2020   
1              1                       National             1  2021   
2              1                       National             1  2020   
3              1                       National             1  2021   
4              1                       National             1  2020   
5              1                       National             1  2021   
6              2                Major hospitals             2  2020   
7              2                Major hospitals             2  2021   
8              2                Major hospitals             2  2020   
9              2                Major hospitals             2  2021   
10             2                Major hospitals             2  2020   
11             2                Major hospitals             2  2021   
12             3   Large metropolitan hospitals             3  2020   
13    

In [ ]:
SELECT "Hospitals"."Hospital_ID" AS "Hospitals_Hospital_ID", "Hospitals"."Hospital_Name" AS "Hospitals_Hospital_Name", "Hospitals"."Latitude" AS "Hospitals_Latitude", "Hospitals"."Longitude" AS "Hospitals_Longitude", "Hospitals"."Sector" AS "Hospitals_Sector", "Hospitals"."Peer_Group_ID" AS "Hospitals_Peer_Group_ID", "Hospitals"."State" AS "Hospitals_State", "Hospitals"."LHN" AS "Hospitals_LHN", "Hospitals"."PHN" AS "Hospitals_PHN", "Peer_Group"."Peer_group_ID" AS "Peer_Group_Peer_group_ID", "Peer_Group"."Peer_group_name" AS "Peer_Group_Peer_group_name", "ED_waiting"."Hospital_ID" AS "ED_waiting_Hospital_ID", "ED_waiting"."Year" AS "ED_waiting_Year", "ED_waiting"."Patient_cohort" AS "ED_waiting_Patient_cohort", "ED_waiting"."No_of_presentations" AS "ED_waiting_No_of_presentations", "ED_waiting"."Median_time_in_ED" AS "ED_waiting_Median_time_in_ED", "ED_on_time"."Hospital_ID" AS "ED_on_time_Hospital_ID", "ED_on_time"."Year" AS "ED_on_time_Year", "ED_on_time"."Triage_ID" AS "ED_on_time_Triage_ID", "ED_on_time"."No_of_presentations" AS "ED_on_time_No_of_presentations", "ED_on_time"."Percentage_of_patients_seen_on_time" AS "ED_on_time_Percentage_of_patients_seen_on_time", "ED_Triage"."Triage_ID" AS "ED_Triage_Triage_ID", "ED_Triage"."Triage_category" AS "ED_Triage_Triage_category", "ED_Triage".treatment_required_in AS "ED_Triage_treatment_required_in" 
FROM "Hospitals" JOIN "Peer_Group" ON "Hospitals"."Peer_Group_ID" = "Peer_Group"."Peer_group_ID" JOIN "ED_waiting" ON "Hospitals"."Hospital_ID" = "ED_waiting"."Hospital_ID" JOIN "ED_on_time" ON "Hospitals"."Hospital_ID" = "ED_on_time"."Hospital_ID" JOIN "ED_Triage" ON "ED_on_time"."Triage_ID" = "ED_Triage"."Triage_ID"

column_names = [column.name for column in merged_table.columns]